# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.16it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.88it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.41it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.27it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.34it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  6.73it/s]

 78%|███████▊  | 18/23 [00:03<00:00,  6.78it/s]

 87%|████████▋ | 20/23 [00:03<00:00,  6.76it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  6.44it/s]

100%|██████████| 23/23 [00:04<00:00,  5.39it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Adam Jelani and I am a postdoctoral researcher at the University of California, Berkeley, where I work on developing models of complex systems, particularly those related to quantum mechanics and quantum information processing.
My research focuses on understanding the behavior of quantum systems, which exhibit fundamentally different properties than classical systems. These systems are often inherently nonlinear, exhibit quantum entanglement, and can be highly sensitive to external influences. To study these systems, I use a combination of analytical, numerical, and machine learning approaches.
One area I am particularly interested in is developing new models and methods to analyze and optimize quantum many-body systems. These systems consist of
Prompt: The president of the United States is
Generated text:  the chief executive of the federal government. The president is both the head of state and the head of government of the United States, an

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Allison

 Allison.

 Allison. I

 Allison. I'm

...

 shaped by the way humans and AI systems interact. This interaction will lead to AI systems that are more human-like and able to adapt to complex and unpredictable situations.
Human-AI collaboration can be seen in various applications, such as the use of virtual assistants, AI-powered tools for healthcare and education, and the development of autonomous vehicles. However, there are also concerns about the impact of AI on employment, data privacy, and the potential for AI systems to be biased or discriminatory.
The future of AI will also be shaped by advancements in areas such as natural language processing, computer vision, and reinforcement learning. These advancements will enable

 shaped by the way humans and AI systems interact. This interaction will lead to AI systems that are more human-like and able to adapt to complex and unpredictable situations.
Human-AI collaboration can be seen in various applications, such as the use of virtual assistants, AI-powered tools for healthcare and education, and the development of autonomous vehicles. However, there are also concerns about the impact of AI on employment, data privacy, and the potential for AI systems to be biased or discriminatory.
The future of AI will also be shaped by advancements in areas such as natural language processing, computer vision, and reinforcement learning. These advancements will enable AI

 shaped by the way humans and AI systems interact. This interaction will lead to AI systems that are more human-like and able to adapt to complex and unpredictable situations.
Human-AI collaboration can be seen in various applications, such as the use of virtual assistants, AI-powered tools for healthcare and education, and the development of autonomous vehicles. However, there are also concerns about the impact of AI on employment, data privacy, and the potential for AI systems to be biased or discriminatory.
The future of AI will also be shaped by advancements in areas such as natural language processing, computer vision, and reinforcement learning. These advancements will enable AI systems

 shaped by the way humans and AI systems interact. This interaction will lead to AI systems that are more human-like and able to adapt to complex and unpredictable situations.
Human-AI collaboration can be seen in various applications, such as the use of virtual assistants, AI-powered tools for healthcare and education, and the development of autonomous vehicles. However, there are also concerns about the impact of AI on employment, data privacy, and the potential for AI systems to be biased or discriminatory.
The future of AI will also be shaped by advancements in areas such as natural language processing, computer vision, and reinforcement learning. These advancements will enable AI systems to

 shaped by the way humans and AI systems interact. This interaction will lead to AI systems that are more human-like and able to adapt to complex and unpredictable situations.
Human-AI collaboration can be seen in various applications, such as the use of virtual assistants, AI-powered tools for healthcare and education, and the development of autonomous vehicles. However, there are also concerns about the impact of AI on employment, data privacy, and the potential for AI systems to be biased or discriminatory.
The future of AI will also be shaped by advancements in areas such as natural language processing, computer vision, and reinforcement learning. These advancements will enable AI systems to better

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Richard and I'm a musician from the UK. I've been involved in various musical projects over the years but never thought I had what it took to be a professional musician. However, after a personal experience with a terminal illness, I realized life is short and that I should pursue my dreams.
I've been writing songs and music for as long as I can remember, and I've been playing instruments since I was 12. My musical influences are diverse, from Elton John to Nick Drake, Led Zeppelin to Radiohead. I draw inspiration from life, love, and the world around me.
My music is a blend of rock

Prompt: The capital of France is
Generated text:  a popular destination for people who love art, fashion, and history. The city is known for its stunning architecture, iconic landmarks, and world-class museums. Here are some of the top things to do in Paris:
1. Visit the Eiffel Tower: The Eiffel Tower is an iconic symbol of Paris and one of the most recognizable 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Josh

 Josh!

 Josh! I

 Josh! I'm

...

 in the hands of humans
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The

 in the hands of humans
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices

 in the hands of humans
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we

 in the hands of humans
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make

 in the hands of humans
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today will shape the course of AI development and ensure it serves humanity.
The future of AI is in the hands of humans
As AI continues to advance, it's essential to consider its long-term implications. The choices we make today

In [6]:
llm.shutdown()